In [9]:
########################################################################################################
# Versión 1
# Generar modelos por estación y atributos seleccionados
# Modelo a utilizar LSTM
# SEGUN LOS "n" DIAS PREVIOS (VENTANA DE "n" VALORES), PREDICE los "ventSalida" DÍAS SIGUIENTES
# del atributo seleccionado
# Guarda los modelos generados con el formato: Modelo_ + COD_ESTACION + ATRIBUTO   ej.: Modelo_46250048_PM10.h5
########################################################################################################

In [10]:
# Parámetros del algoritmo

# Definir el número de días de la ventana de entrada (días previos a considerar para la predicción)
n = 30

# Definir el número de días a predecir por cada ventana de entrada
ventSalida = 1

# seleccionar lista de atributos a generar modelo de predicción
atributos = ['PM10', 'O3']

# Definir número de epoch máximo para entrenamiento
max_epochs = 50

In [11]:
# librerías necesarias

import pandas as pd
import numpy as np
import plotly 
import plotly.express as px
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

np.random.seed(100)
tf.random.set_seed(100)

# Configuración general de las figuras que representaremos
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (20, 10)

import warnings
warnings.filterwarnings("ignore") # specify to ignore warning messages

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
path = '/content/drive/MyDrive/datoscsv/'
dfCompleto = pd.read_csv(path + 'Valencia_SeleccionDatos_ParaModelar_v1.csv', sep=';', index_col='fecha', parse_dates=['fecha'])

# Recuperar la fecha como campo
dfCompleto.insert(0, 'FECHA', dfCompleto.index.strftime('%Y-%m-%d'))


In [14]:
def procesar_atributo(atributo, cod_esta, nom_esta, df):
  print(f'Procesando atributo: {atributo}')
  print()
  # eliminar filas sin el atributo para probar
  df = df[df[atributo].notnull()]
  # Vamos a coger hasta el 2018 para entrenamiento y los datos del 2019 para testear
  # Para poder introducir los datos en una red neuronal hay que transformarlos, ya que solo funciona con arrays
  df2 = df[[atributo]]
  # print('df2:', df2.describe().transpose())
  train = df2[:'2018']    # guardar desde fecha más antigua hasta final de 2018
  test = train.iloc[-n:]  # guardar los "n" últimos dias del 2018
  test = test.append(df2['2019'], ignore_index=False) # añadir a los "n" días del 2018 + el 2019 completo

  train[atributo].plot(legend=True)
  test[atributo].plot(legend=True)
  plt.legend(['Train (inicio-2018)', 'Test (2019)'])
  plt.title(nom_esta + ': ' + atributo )
  plt.show()
  print()
  
  # normalizar datos
  sc = MinMaxScaler(feature_range=(0,1))
  train_scaled = sc.fit_transform(train)

  # Preparar ventana de entrada
  # Hay que crear una estructura de datos para indicarle a la red lo que tiene que recordar 
  # y poder hacer una predección en base a datos anteriores
  X_train = []
  y_train = []

  # Estructura de datos con "n" pasos y uno o mas valores de salida
  # "n" timesteps significa que para un día dado la red es capaz de mirar los "n" días anteriores:
  # "n" valores anteriores al día actual y en base a esa información predice los "ventSalida" dias futuros 
  for i in range(n,len(train_scaled)):
      # X: para cada día hacemos un bloque correspondiente al valor de los "n" días anteriores
      X_train.append(train_scaled[i-n:i,0])
      # Y: el valor del día
      y_train.append(train_scaled[i,0])

  # Tenemos una matriz donde cada fila contiene "n" columnas correspondientes a los "n" días anteriores
  X_train, y_train = np.array(X_train), np.array(y_train)

  # Reshape X_train para que se ajuste al modelo en Keras
  # añadiendo una nueva dimensión 
  X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

  # Modelo LSTM
  # Creamos un modelo secuencial al que iremos añadiendo capas, que pueden ser:

  # Dense: Capa de neuronas artificiales
  # LSTM: Capa de neuronas con capacidad de memoria
  # Dropout: Es una técnica de regularización que elimina conexiones neuronales para evitar el sobreajuste, de esta forma se evita
  #          que el algoritmo memorice los datos en vez de aprender de ellos.
  # Antes de crear la Red LSTM debemos reajustar los sets que acabamos de obtener, para indicar que cada ejemplo de entrenamiento a
  #          la entrada del modelo será un vector de "n" x 1.
  # Para el caso de cada una de las salidas (almacenadas en y_train) debemos simplemente especificar que su tamaño, que será igual
  #           al número de valores predichos por la red neuronal.
  # Definimos el tamaño de los datos de entrada y de salida
  dim_entrada = (X_train.shape[1],1)
  dim_salida = ventSalida

  modelo = Sequential()
  # Añadimos la primera capa y especificamos el número de neuromas (50) y el tamaño de cada dato de entrada
  modelo.add(LSTM (units=50, return_sequences=True, input_shape=dim_entrada))
  modelo.add(Dropout(0.2))
  # Añadimos la segunda capa
  modelo.add(LSTM (units=50, return_sequences=True))
  modelo.add(Dropout(0.2))
  # Añadimos la tercera capa
  modelo.add(LSTM (units=50, return_sequences=True))
  modelo.add(Dropout(0.2))
  # Añadimos la cuarta capa
  modelo.add(LSTM (units=50))
  modelo.add(Dropout(0.2))
  # Añadimos la capa de salida (Dense) y especificamos que el campo de salida es igual a dim_salida
  modelo.add(Dense(units=dim_salida))

  # Compilación
  # El optimizador se encarga de optimizar y actualizar los pesos de la red. Usaremos el algoritmo adam que es el que mejores resultados nos aporta.
  # loss: Cuando la red hace una predicción coge el valor real y lo compará con la predicción. Para calcular el error de predicción, se utilizará el método error cuadrático medio.
  
  # Compilamos el modelo definiendo la función de error (loss) y el método que se usa para minimizarla (optimizer)
  modelo.compile(optimizer='adam', loss='mean_squared_error')

  # Entrenamiento
  # epochs: Especificamos las veces que la red propagara el error hacia atrás para aprender y hacer mejores predicciones. Es la cantidad de veces que la red realizará una predicción y propagara hacia atrás el error, para aprender de él y hacer mejores predicciones
  # batch_size: Para este proceso de predicción, corrección y propagación hacia atrás, no lo haremos con un solo bloque de datos, si no de 32. Si no el entrenamiento sería demasiado pesado para la red, el tener que actualizar los pesos a cada dato. Vamos actualizando los pesos con bloques de datos.

  # Entrenamos el modelo con max_epochs (iteraciones) y batch_size=32 (lotes de 32 datos)

  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

  modelo.fit(X_train,y_train,epochs=max_epochs, batch_size=32, callbacks=[callback])

  # Predicciones
  # Hay que preparar los datos como en el caso de train, esto es, hacemos bloques de "n" días para predecir el valor de la acción en el siguiente día.
  test_scaled = sc.fit_transform(test)

  X_test = []
  for i in range(n,len(test_scaled)):
      X_test.append(test_scaled[i-n:i,0])

  X_test = np.array(X_test)
  X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

  # Realizamos la prediccion y aplicamos normalización inversa para que esté en escala real
  prediccion = modelo.predict(X_test)
  prediccion = sc.inverse_transform(prediccion)

  # Visualización del resultado
  resultados = test.tail(len(test)-n)
  resultados['Prediccion'] = prediccion

  # print(resultados)

  plt.plot(train[atributo], color='green')
  plt.plot(resultados[atributo], color='red', label='Valor real de ' + atributo)
  plt.plot(resultados['Prediccion'], color='blue', label='Predicción de ' + atributo)
  plt.xlabel('Tiempo')
  plt.ylabel('Valor de ' + atributo)
  plt.legend()
  plt.title(nom_esta + ': ' + atributo )
  plt.show()
  print()

  plt.plot(resultados[atributo], color='red', label='Valor real de ' + atributo)
  plt.plot(resultados['Prediccion'], color='blue', label='Predicción de ' + atributo)
  plt.xlabel('Tiempo')
  plt.ylabel('Valor de ' + atributo)
  plt.legend()
  plt.title(nom_esta + ': ' + atributo )
  plt.show()
  print()

  from sklearn.metrics import mean_absolute_error
  from sklearn.metrics import mean_squared_error
  from math import sqrt

  # ratios de la predicción
  mae = mean_absolute_error(resultados['Prediccion'], resultados[atributo])
  rmse = sqrt(mean_squared_error(resultados['Prediccion'], resultados[atributo]))
  
  print('Modelo:' , cod_esta, '-', atributo )
  print('Error cuadrático medio: %.3f' % rmse)
  print('  Error absoluto medio: %.3f' % mae)
  print('========================================')
  print()

  # Guardar el Modelo
  pathModelos = '/content/drive/MyDrive/modelosGuardados/'
  modelo.save(pathModelos + 'Modelo_' + str(cod_esta) + '_' + atributo + '.h5')

  # Recrea exactamente el mismo modelo solo desde el archivo:
  # new_model = keras.models.load_model('nombremodelo.h5')

    

In [15]:
def procesar_estacion(cod_esta, nom_esta, df):
  print(f'Procesando estacion: {cod_esta} - {nom_esta}')
  print()
  # seleccionar los datos de la estacion a procesar
  df = df[df['COD_ESTACION']==cod_esta]
  # print(df)
  # procesar los atributos de la estación
  for atrib in atributos:
    procesar_atributo(atrib, cod_esta, nom_esta, df)

In [16]:
# Proceso principal

# print(dfCompleto['COD_ESTACION'].unique())
# print(dfCompleto['NOM_ESTACION'].unique())
cod_estaciones = dfCompleto['COD_ESTACION'].unique()
nom_estaciones = dfCompleto['NOM_ESTACION'].unique()
for cod_esta, nom_esta in zip(cod_estaciones, nom_estaciones):
  procesar_estacion(cod_esta, nom_esta, dfCompleto)

Output hidden; open in https://colab.research.google.com to view.